## 正向神经网络

In [2]:
import pandas as pd
df = pd.read_csv('./ccf_offline_stage1_train.csv')
df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


使用Pytorch或者Tensorflow，搭建三层神经网络模型，对用户的促销券使用情况建模，并在随机切分的测试集上查看训练准确率，并调优模型并和XGBoost模型结果对比
#### 该模型质量劣于XGBoost模型是正常情况

In [3]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
df['Discount_rate']=df['Discount_rate'].fillna(0)
df['Discount_rate_new']=df['Discount_rate'].apply(lambda x:x if ':' not in str(x) else ((float(x.split(':')[0])-float(x.split(':')[1]))/float(x.split(':')[0])))
df['Discount_rate_new']=df['Discount_rate_new'].apply(lambda x:round(float(x),2))
from sklearn.model_selection import train_test_split
df['label']=df['Date'].apply(lambda x:1 if x==x else 0)
y=df['label']
from sklearn.model_selection import train_test_split
X=df
df['label']=df['Date'].apply(lambda x:1 if x==x else 0)
y=df['label']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)
X_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Discount_rate_new,label
382193,2462193,5341,7751.0,50:10,0.0,20160131.0,NaN,0.80,0
1250747,3071464,6485,2079.0,30:5,0.0,20160524.0,20160529.0,0.83,1
580400,1104814,5755,13465.0,100:20,1.0,20160416.0,NaN,0.80,0
1356874,1898571,3561,14006.0,10:1,0.0,20160115.0,NaN,0.90,0
78932,1197041,2247,98.0,30:5,10.0,20160522.0,NaN,0.83,0


In [7]:
from sklearn.preprocessing import StandardScaler
feature=['Merchant_id','Coupon_id','Distance','Discount_rate_new']
scaler=StandardScaler()
scaler.fit(X_train.loc[:,feature])
X_train_scaler=pd.DataFrame(scaler.transform(X_train.loc[:,feature]),columns=feature)
X_train_scaler=X_train_scaler.fillna(X_train_scaler.mean())


In [9]:
X_test_scaler=pd.DataFrame(scaler.transform(X_test.loc[:,feature]),columns=feature)
X_test_scaler=X_test_scaler.fillna(X_test_scaler.mean())
X_test_scaler.head()

,Merchant_id,Coupon_id,Distance,Discount_rate_new
0,0.042725,-0.000621,-0.390638,-1.207849
1,0.983392,0.775011,2.193110,0.939493
2,-1.013267,-0.422319,-0.677721,0.820196
3,0.389114,-0.000621,1.044778,-1.207849
4,1.221021,-0.000621,-0.677721,-1.207849


In [10]:
class Torch_Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.data = df.values
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    
X_train_scaler['label']=y_train.values    
X_test_scaler['label']=y_test.values

trainset = Torch_Dataset(X_train_scaler)
testset = Torch_Dataset(X_test_scaler)

batch_size = 10000
kwargs = {'num_workers': 0, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = True, **kwargs)

In [11]:
class o2o_nn(nn.Module):
    def __init__(self):
        super(o2o_nn, self).__init__()

        self.fc1 = nn.Linear(4,3)
        self.fc2 = nn.Linear(3,3)
        self.fc3 = nn.Linear(3,1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        h1 = self.relu(self.fc1(x))
        h1 = self.dropout(h1)
        h2 = self.relu(self.fc2(h1))
        h2 = self.dropout(h2)
        h3 = self.relu(self.fc3(h2))
        h3 = self.dropout(h3)
        return self.sigmoid(h3)
model =o2o_nn()
model = model.float()

In [12]:
loss_function = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [13]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        feature = data[:,:-1]
        label = data[:,-1]
        data = Variable(feature)
        optimizer.zero_grad()
        label_pred = model(feature.float())
        loss = loss_function(label_pred.float(), label.view(-1,1).float())
        loss.backward()
        train_loss += loss.data
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data / len(data)))       

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [14]:
EPOCHS = 10
for epoch in range(1,EPOCHS+1):
    train(epoch)
    

Train Epoch: 1 [0/1228418 (0%)]	Loss: 0.000069
Train Epoch: 1 [100000/1228418 (8%)]	Loss: 0.000069
Train Epoch: 1 [200000/1228418 (16%)]	Loss: 0.000069
Train Epoch: 1 [300000/1228418 (24%)]	Loss: 0.000069
Train Epoch: 1 [400000/1228418 (33%)]	Loss: 0.000069
Train Epoch: 1 [500000/1228418 (41%)]	Loss: 0.000069
Train Epoch: 1 [600000/1228418 (49%)]	Loss: 0.000069
Train Epoch: 1 [700000/1228418 (57%)]	Loss: 0.000069
Train Epoch: 1 [800000/1228418 (65%)]	Loss: 0.000068
Train Epoch: 1 [900000/1228418 (73%)]	Loss: 0.000068
Train Epoch: 1 [1000000/1228418 (81%)]	Loss: 0.000068
Train Epoch: 1 [1100000/1228418 (89%)]	Loss: 0.000068
Train Epoch: 1 [1200000/1228418 (98%)]	Loss: 0.000068
====> Epoch: 1 Average loss: 0.0001
Train Epoch: 2 [0/1228418 (0%)]	Loss: 0.000067
Train Epoch: 2 [100000/1228418 (8%)]	Loss: 0.000067
Train Epoch: 2 [200000/1228418 (16%)]	Loss: 0.000067
Train Epoch: 2 [300000/1228418 (24%)]	Loss: 0.000067
Train Epoch: 2 [400000/1228418 (33%)]	Loss: 0.000067
Train Epoch: 2 [50000

In [16]:
model.eval()
test_loss = 0
y_tests = []
y_preds = []
for data in test_loader:
    pred_list = []
    feature = data[:,:-1]
    label = data[:,-1]

    label_pred = model(feature.float())
    y_preds += label_pred.detach().numpy().ravel().tolist()
    y_tests += label.numpy().ravel().tolist()

In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_tests, y_preds)

0.9520554687350172

### XGBoost

In [32]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score
df=pd.read_csv('ccf_offline_stage1_train.csv')
df['Discount_rate']=df['Discount_rate'].fillna(0)
df['Discount_rate_new']=df['Discount_rate'].apply(lambda x:x if ':' not in str(x) else ((float(x.split(':')[0])-float(x.split(':')[1]))/float(x.split(':')[0])))
df['Discount_rate_new']=df['Discount_rate_new'].apply(lambda x:round(float(x),2))
from sklearn.model_selection import train_test_split
df['label']=df['Date'].apply(lambda x:1 if x==x else 0)
y=df['label']
from sklearn.model_selection import train_test_split
X=df
df['label']=df['Date'].apply(lambda x:1 if x==x else 0)
y=df['label']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)
feature=['Merchant_id','Coupon_id','Distance','Discount_rate_new']

from sklearn.preprocessing import StandardScaler
feature=['Merchant_id','Coupon_id','Distance','Discount_rate_new']
scaler=StandardScaler()
scaler.fit(X_train.loc[:,feature])
X_train_scaler=pd.DataFrame(scaler.transform(X_train.loc[:,feature]),columns=feature)
X_train_scaler=X_train_scaler.fillna(X_train_scaler.mean())
X_test_scaler=pd.DataFrame(scaler.transform(X_test.loc[:,feature]),columns=feature)
X_test_scaler=X_test_scaler.fillna(X_test_scaler.mean())
test_data=xgb.DMatrix(X_test_scaler.loc[:,feature].values,label=y_test)
xgb_params={
    'eta':0.3,
    'silent':True,
    'objective':'multi:softprob',
    'num_class':3,
    'max_depth':3
}
num_round=20
model=xgb.train(xgb_params,xgb.DMatrix(X_train_scaler.loc[:,feature].values,label=y_train),num_round)
test_pre=model.predict(test_data)
test_pre_1=np.asarray([np.argmax(row) for row in test_pre])
print(roc_auc_score(y_test,test_pre_1,average='macro'))

D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.9520586794544563


这次的作业就到这里了！祝大家学习进步！
![image alt <](http://5b0988e595225.cdn.sohucs.com/images/20190420/1d1070881fd540db817b2a3bdd967f37.gif)